# Ensemble Learning

## Initial Imports

In [72]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [74]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [75]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [76]:
df.shape

(68817, 86)

In [43]:
df['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

Loan status column is still the positive event. 

## Split the Data into Training and Testing

In [99]:
# Create our features
X = df.copy()                                         # copy source df so we can drop columns unlrelated to our features. 
X.drop(columns='loan_status', inplace=True)           # our features are everything EXCEPT loan status
X = pd.get_dummies(X)

# Create our target
y = df['loan_status']                                 # Why do we not have to do the .values.reshape(-1, 1) here? 

In [100]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [101]:
y[:5]

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [80]:
# Check the balance of our target values
# YOUR CODE HERE
y['loan_status'].value_counts()                                              # Code fails?? I'm not understanding why this cell is kicking back an error.
                                                                             # we set our y-vector to the df['loan_status'] should be very simple and straight forward. 

KeyError: 'loan_status'

In [103]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [104]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [105]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [106]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [107]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier                     # Import Balanced Random Forest Classifier module

balanced_forest_model = BalancedRandomForestClassifier(
    n_estimators=100,                                                              # I am setting n_estimators to 100 per Instructions. 
    random_state=1                                                                 # set random_state to 1. 
)    

# Fit forest model with taining data.
balanced_forest_model = balanced_forest_model.fit(X_train_scaled, y_train)                      

In [108]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
predictions = balanced_forest_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.7887512850910909

In [109]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm,
    index = ['Actual 0', 'Actual 1'],
    columns = ['Predicted 0', 'Predicted 1'] 
)
cm

array([[   71,    30],
       [ 2146, 14958]], dtype=int64)

**ANALYSIS  of TEST results:**
* Actual 0 (actual low_risk applicants)
* True Negatives = 71
* False Positives = 30
* Of the 101 applicants that were actually false, or low_risk, we correctly predicted 70% to be low_risk.
* Of our actual 101 low_risk applications in the test dataset, we incorrectly labeled 30 applicants as high risk.

* Actual 1 (actual high_risk applicants)
* False Negatives = 2146
* True Postives = 14,958
* Our test data for the model is tremendously weighted towards high_risk applicants.
* We incorrecly labeled 2146 high_risk applicants as low_risk. That's 12.5% of total high_risk applicants in our test dataset.

In [110]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.63     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.63     17205



**ANALYSIS:**
* High Risk
* Precision score is low. .03%
* We predicted 14,988 applicants in the test data to be high_risk. Only 30 False Positves were predicted wrong. So 99% of our high risk were classified correctly, or 14,958 True Positives out of total 14,988 total predicted. 
* Precision score should be much higher with such low False Positives.
* For the actual high risk we classified 14,958 True Positives, or 87% correct. 
* 2146 is a lot of False Negatives, and it should hit our recall score. So a 70% is a little ok for my expectations.


* Low Risk
* Precision score 100% is not right considering we did have False Positives. 100% precision means all Positives are True Positives.

My analysis is that our balanced forest model is not capturing enough variables?

In [111]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = balanced_forest_model.feature_importances_

importances_sorted = sorted(zip(balanced_forest_model.feature_importances_, X.columns), reverse=True)

importances_sorted[:10]

[(0.07876809003486353, 'total_rec_prncp'),
 (0.05883806887524815, 'total_pymnt'),
 (0.05625613759225244, 'total_pymnt_inv'),
 (0.05355513093134745, 'total_rec_int'),
 (0.0500331813446525, 'last_pymnt_amnt'),
 (0.02966959508700077, 'int_rate'),
 (0.021129125328012987, 'issue_d_Jan-2019'),
 (0.01980242888931366, 'installment'),
 (0.01747062730041245, 'dti'),
 (0.016858293184471483, 'out_prncp_inv')]

The top feature is 'total_rec_prncp' at .08%. Followed by total_pymnt at .06% 

### Easy Ensemble Classifier

In [94]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier

easy_ensemble_model = EasyEnsembleClassifier(
    n_estimators = 100,
    random_state=1
)
easy_ensemble_model.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [96]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
predictions_ensem = easy_ensemble_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions_ensem)

0.931601605553446

In [97]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, predictions_ensem)

array([[   93,     8],
       [  985, 16119]], dtype=int64)

In [98]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions_ensem))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.\
    
    **Easy Ensemble Model accuracy score was 93%**
    **Balanced Random Forest Model accuracy score was 79%**

2. Which model had the best recall score?

    YOUR ANSWER HERE.\
    
    **Easy Ensemble Model was better. We had recall rates over 90%, which means this model is good at classifying the actual events. We had a much lower occurrence of False Negatves in the 
    model at 985 vs the 2146 we had with the Balanced Forest, so recall should improve. We got better at classifying high risk applicants.**

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.\
    
    **Easy Esemble model at 93%. Balanced Forest was at 78%**

4. What are the top three features?

    YOUR ANSWER HERE.
    
    **The top 3 features are:** 
    * total principle received = .08%
    * total payment = .06%
    * total invoice = .06%